In [1]:
import requests
import pandas as pd
import os
import re


In [2]:
df_vitoria = pd.read_csv("data/Datos_Aemet/vitoria_csv/datos_vitoria.csv", sep =",")
df_vitoria

,fecha,tm_mes,tm_max,tm_min,ta_max,ta_min,ti_max,ts_min,p_mes,np_300,nt_30,inso
0,1970-1,6.6,9.8,3.5,14.6(09),-6.4(07),2.6,8.4,127.2,0.0,0.0,2.4
1,1970-2,4.9,8.6,1.2,13.8(22),-7.0(28),0.6,7.2,136.4,1.0,0.0,3.8
2,1970-3,5.2,9.0,1.4,20.0(21),-4.0(12),1.6,9.2,55.2,0.0,0.0,3.8
3,1970-4,8.4,13.5,3.3,28.4(17),-2.6(06),5.6,9.2,37.4,0.0,0.0,5.9
4,1970-5,12.3,17.5,7.0,30.2(24),0.4(02),10.0,11.6,93.9,2.0,1.0,6.9
...,...,...,...,...,...,...,...,...,...,...,...,...
684,2022-9,17.2,24.8,9.5,34.5(11),2.5(22),15.0,20.2,17.9,0.0,6.0,6.6
685,2022-10,17.4,24.3,10.5,28.1(05),5.0(01),19.0,15.0,31.9,0.0,0.0,5.3
686,2022-11,9.9,15.2,4.5,20.5(17),-0.7(27),8.0,11.2,75.7,0.0,0.0,3.5
687,2022-12,8.8,12.4,5.1,18.8(31),-1.2(05),5.5,12.9,28.6,0.0,0.0,2.9


In [3]:
#Separamos la columna "fecha" con año-mes en dos
df_vitoria[['año', 'mes']] = df_vitoria['fecha'].str.split('-', expand=True)
# Eliminamos la columna "fecha"
df_vitoria2 = df_vitoria.drop(columns=["fecha"], axis=1)
# Ponemos las nuevas columnas año, mes las primeras
column_order = ['año', 'mes'] + df_vitoria2.columns.tolist()[:-2]
df_vitoria3 = df_vitoria2.reindex(columns=column_order)
df_vitoria3
df_vitoria3['ta_max'] = df_vitoria3['ta_max'].str.replace(r'\(.*\)', '', regex=True)
df_vitoria3['ta_min'] = df_vitoria3['ta_min'].str.replace(r'\(.*\)', '', regex=True)
df_vitoria3
# Así se pasan las filas de los datos anuales a otro dataframe
df_vitoria_anual = pd.DataFrame(df_vitoria3.iloc[12::13])
df_vitoria_anual
# Y eliminamos esas filas del dataframe original
df_vitoria_final = df_vitoria3.drop(range(12,680,13), axis=0)
df_vitoria_final
#Generamos archivos csv:
df_vitoria_anual.to_csv('data/csv_creados/vitoria_anual.csv', index=False)
df_vitoria_final.to_csv('data/csv_creados/vitoria_final.csv', index=False)






In [4]:
df_vitoria_final.info()

<class 'pandas.core.frame.DataFrame'>
Index: 637 entries, 0 to 688
Data columns (total 13 columns):
 #   Column  Non-Null Count  Dtype  
---  ------  --------------  -----  
 0   año     637 non-null    object 
 1   mes     637 non-null    object 
 2   tm_mes  633 non-null    float64
 3   tm_max  633 non-null    float64
 4   tm_min  633 non-null    float64
 5   ta_max  633 non-null    object 
 6   ta_min  633 non-null    object 
 7   ti_max  633 non-null    float64
 8   ts_min  633 non-null    float64
 9   p_mes   633 non-null    float64
 10  np_300  633 non-null    float64
 11  nt_30   633 non-null    float64
 12  inso    628 non-null    float64
dtypes: float64(9), object(4)
memory usage: 69.7+ KB


In [8]:
#Tiene 4 columnas con valores nulos que eliminaremos. 
df_vitoria_final.reset_index(drop=True, inplace=True)
df_vitoria_final_sinull= df_vitoria_final.drop([118,119,120,121])


In [17]:
df_vitoria_final_sinull['inso'] = df_vitoria_final_sinull['inso'].fillna(0)
df_vitoria_final_sinull.reset_index(drop=True, inplace=True)
df_vitoria_final_sinull.to_csv('data/csv_creados/vitoria_final_sinull.csv', index=False)
df_vitoria_final_sinull

,año,mes,tm_mes,tm_max,tm_min,ta_max,ta_min,ti_max,ts_min,p_mes,np_300,nt_30,inso
0,1970,1,6.6,9.8,3.5,14.6,-6.4,2.6,8.4,127.2,0.0,0.0,2.4
1,1970,2,4.9,8.6,1.2,13.8,-7.0,0.6,7.2,136.4,1.0,0.0,3.8
2,1970,3,5.2,9.0,1.4,20.0,-4.0,1.6,9.2,55.2,0.0,0.0,3.8
3,1970,4,8.4,13.5,3.3,28.4,-2.6,5.6,9.2,37.4,0.0,0.0,5.9
4,1970,5,12.3,17.5,7.0,30.2,0.4,10.0,11.6,93.9,2.0,1.0,6.9
...,...,...,...,...,...,...,...,...,...,...,...,...,...
628,2022,9,17.2,24.8,9.5,34.5,2.5,15.0,20.2,17.9,0.0,6.0,6.6
629,2022,10,17.4,24.3,10.5,28.1,5.0,19.0,15.0,31.9,0.0,0.0,5.3
630,2022,11,9.9,15.2,4.5,20.5,-0.7,8.0,11.2,75.7,0.0,0.0,3.5
631,2022,12,8.8,12.4,5.1,18.8,-1.2,5.5,12.9,28.6,0.0,0.0,2.9


In [10]:
df_vitoria_final_sinull.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 633 entries, 0 to 632
Data columns (total 13 columns):
 #   Column  Non-Null Count  Dtype  
---  ------  --------------  -----  
 0   año     633 non-null    object 
 1   mes     633 non-null    object 
 2   tm_mes  633 non-null    float64
 3   tm_max  633 non-null    float64
 4   tm_min  633 non-null    float64
 5   ta_max  633 non-null    object 
 6   ta_min  633 non-null    object 
 7   ti_max  633 non-null    float64
 8   ts_min  633 non-null    float64
 9   p_mes   633 non-null    float64
 10  np_300  633 non-null    float64
 11  nt_30   633 non-null    float64
 12  inso    633 non-null    float64
dtypes: float64(9), object(4)
memory usage: 64.4+ KB


In [12]:
df_vitoria_anual.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 53 entries, 0 to 52
Data columns (total 13 columns):
 #   Column  Non-Null Count  Dtype  
---  ------  --------------  -----  
 0   año     53 non-null     object 
 1   mes     53 non-null     object 
 2   tm_mes  52 non-null     float64
 3   tm_max  52 non-null     float64
 4   tm_min  52 non-null     float64
 5   ta_max  52 non-null     object 
 6   ta_min  52 non-null     object 
 7   ti_max  52 non-null     float64
 8   ts_min  52 non-null     float64
 9   p_mes   52 non-null     float64
 10  np_300  52 non-null     float64
 11  nt_30   52 non-null     float64
 12  inso    50 non-null     float64
dtypes: float64(9), object(4)
memory usage: 5.5+ KB


In [16]:
#Tiene 2 filas  con valores nulos que eliminaremos que sustituimos por la anterior y la siguiente. 
df_vitoria_anual.reset_index(drop=True, inplace=True)
df_vitoria_anual.iloc[9] = df_vitoria_anual.iloc[8]
df_vitoria_anual.iloc[10] = df_vitoria_anual.iloc[11]
df_vitoria_anual.loc[9,"año"]=1979
df_vitoria_anual.loc[10,"año"]=1980

In [12]:
df_vitoria_anual.head()

,año,mes,tm_mes,tm_max,tm_min,ta_max,ta_min,ti_max,ts_min,p_mes,np_300,nt_30,inso
12,1970,13,11.2,16.2,6.2,35.8,-12.4,-2.2,18.8,783.1,4.0,18.0,5.6
25,1971,13,10.8,15.7,5.8,34.6,-20.8,-5.4,17.6,1023.9,5.0,22.0,5.1
38,1972,13,10.3,14.8,5.8,32.4,-5.0,2.6,17.0,1062.9,2.0,4.0,NaN
51,1973,13,10.8,15.6,6.0,35.8,-6.8,0.0,19.4,875.9,5.0,19.0,5.2
64,1974,13,10.9,15.5,6.2,36.4,-5.6,2.8,18.0,970.5,4.0,16.0,4.7


In [16]:
df_vitoria_anual['inso'] = df_vitoria_anual['inso'].fillna(0)
df_vitoria_anual_sinull1=df_vitoria_anual.drop(["mes"], axis=1)
df_vitoria_anual_sinull1.reset_index(drop=True, inplace=True)
df_vitoria_anual_sinull1.to_csv('data/csv_creados/vitoria_anual_sinull1.csv', index=False)

In [19]:
df_vitoria_anual_sinull1.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 53 entries, 0 to 52
Data columns (total 12 columns):
 #   Column  Non-Null Count  Dtype  
---  ------  --------------  -----  
 0   año     53 non-null     object 
 1   tm_mes  53 non-null     float64
 2   tm_max  53 non-null     float64
 3   tm_min  53 non-null     float64
 4   ta_max  53 non-null     object 
 5   ta_min  53 non-null     object 
 6   ti_max  53 non-null     float64
 7   ts_min  53 non-null     float64
 8   p_mes   53 non-null     float64
 9   np_300  53 non-null     float64
 10  nt_30   53 non-null     float64
 11  inso    53 non-null     float64
dtypes: float64(9), object(3)
memory usage: 5.1+ KB
